In [6]:
#this code is for extracting text for training
# coding=utf-8
import sys
import os.path
import re

import logging
from bs4 import BeautifulSoup
from pdfminer3k.pdfparser import PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed

logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

def prase_all(file_dir, test_file_number):
    pathDir = os.listdir(file_dir)
    file_path_list = list()
    for allDir in pathDir:
        child = os.path.join('%s%s' % (file_dir, allDir))
        file_path_list.append(child)
    for filename in file_path_list:
        #print(filename)
        pdf2TxtManager = CPdf2TxtManager()
        pdf2TxtManager.changePdfToText(filename)
        test_file_number += -1
        if test_file_number == 0:
            break

class CPdf2TxtManager():
    def __init__(self):
        '''''
        Constructor
        '''
    def changePdfToText(self, file_path):
        file = open(file_path, 'rb')
        praser = PDFParser(file)
        file.close()
        doc = PDFDocument()
        praser.set_document(doc)
        doc.set_parser(praser)
        doc.initialize()
        if not doc.is_extractable:
            #raise PDFTextExtractionNotAllowed
            print(file_path)
            return 0
        else:
            rsrcmgr = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(rsrcmgr, laparams = laparams)
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            results = ""
            for page in doc.get_pages(): 
                interpreter.process_page(page)
                layout = device.get_result()
                for x in layout:
                    if hasattr(x, "get_text"):
                        results += x.get_text()
            results = re.sub(r'\s+', '', results)
            #results = results.replace('/n', ' ')
            #print(file_path)
            #print(results)
            txt_path = file_path
            f = open(txt_path.replace('pdf', 'txt'),'wb')
            f.write(bytes(results, encoding = "utf-8"))
            file.close()

if __name__ == '__main__':
    prase_all('F:\\FDDC2018data\\round1_train_20180518\\重大合同\\pdf\\', -1)
    #path = u'F:\\FDDC2018data\\round1_train_20180518\\重大合同\\pdf\\1008828.pdf'
    #pdf2TxtManager = CPdf2TxtManager()
    #pdf2TxtManager.changePdfToText(path)

ModuleNotFoundError: No module named 'pdfminer3k'

In [199]:
# coding=utf-8
import sys
import os.path
import re
import logging
from bs4 import BeautifulSoup
from pdfminer.pdfparser import PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed

logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

def prase_all(file_dir, test_file_number):
    pathDir = os.listdir(file_dir)
    file_path_list = list()
    for allDir in pathDir:
        child = os.path.join('%s%s' % (file_dir, allDir))
        file_path_list.append(child)
    for filename in file_path_list:
        #print(filename)
        pdf2TxtManager = CPdf2TxtManager()
        pdf2TxtManager.changePdfToText(filename)
        test_file_number += -1
        if test_file_number == 0:
            break

class CPdf2TxtManager():
    def __init__(self):
        '''''
        Constructor
        '''
    def changePdfToText(self, file_path):
        file = open(file_path, 'rb')
        praser = PDFParser(file)
        file.close()
        doc = PDFDocument()
        praser.set_document(doc)
        doc.set_parser(praser)
        doc.initialize()
        if not doc.is_extractable:
            #raise PDFTextExtractionNotAllowed
            print(file_path)
            return 0
        else:
            rsrcmgr = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(rsrcmgr, laparams = laparams)
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            results = ""
            for page in doc.get_pages(): 
                interpreter.process_page(page)
                layout = device.get_result()
                for x in layout:
                    if hasattr(x, "get_text"):
                        results += x.get_text()
            #no_space_results = re.sub(r'\s+', '', results)
            #results = results.replace('/n', ' ')
            #print(no_space_results + '\n')
            return results

def load_file(file_name):
    f = open("F:\\FDDC2018data\\round1_train_20180518\\重大合同\\" + file_name + ".txt","rb")   
    lines = f.readlines()
    lines_str = list()
    for item in lines:
        tmp = str(item, encoding = 'utf-8')
        tmp = tmp.replace(' ',' ')
        tmp = tmp.replace(' ','(.+?)')
        tmp = tmp.replace("\r\n", '')
        tmp = tmp.replace("\ufeff", '')
        lines_str.append(tmp)
    return lines_str

def find(keydict, text):
    name = ""
    for item in keydict:
        p = re.compile(r'' + item)
        if len(p.findall(text)):
            if (not ('，' in p.findall(text)[0])) & (not ('。' in p.findall(text)[0])):
                name = p.findall(text)[0]
                break
    return name

def prefix_check(keydict, name):
    for item in keydict:
        if re.match(item, name):
            if re.match(item,name).span(0)[0] == 0:
                name = name.lstrip(item)
                break
    return name

def suffix_check(keydict, name):
    for item in keydict:
        p = re.compile(r'' + item + "$")
        if len(p.findall(name)):
            name = name.rstrip(item)
            break
    return name

def brackets_check(keydict, text):
    name = text
    flag = False
    if re.compile('（(.+?)）').findall(name):
        tmp = re.compile('（(.+?)）').findall(name)
        for item1 in tmp:
            for item2 in keydict:
                if item1.find(item2):
                    p = re.compile('（' + item1 + '）')
                    name = p.sub('', name)
                    flag = True
    if re.compile(r'[(](.+?)[)]').findall(name):
        tmp = re.compile(r'[(](.+?)[)]').findall(name)
        for item1 in tmp:
            for item2 in keydict:
                if item1.find(item2):
                    p = re.compile('[(]' + item1 + '[)]')
                    name = p.sub('', name)
                    flag = True
    if flag == False:
        return text
    else:
        return name

def re_analyse(text):
    no_space_text = re.sub(r'\s+', '', text)
    print(text)
    #id

    #Party A
    party_a = find(party_a_dict, no_space_text)
    
    if len(party_a):
        eng_re = re.compile(r'[a-zA-Z]')#English Company Name
        if len(re.findall(eng_re, party_a)):
            party_a = find(party_a_dict, text)
            party_a = party_a.strip()
        party_a = prefix_check(cprefix, party_a)
        party_a = suffix_check(csuffix, party_a)
        party_a = brackets_check(cbrk, party_a)
    
    print("Party A:")
    print(party_a)
    #Party B
    party_b = find(party_b_dict, no_space_text)
    
    if len(party_b):
        party_b = brackets_check(cbrk, party_b)
    
    print("Party B:")
    print(party_b)
    #Project Name
    project_name = find(project_name_dict, no_space_text)
    
    project_name_list = list()
    if len(project_name):
        project_name_list = re.compile('“(.+?)”').findall(project_name)
    
    print("Project Name:")
    if len(project_name_list):
        for item in project_name_list:
            print(item)
    else:
        print(project_name)
    #print(" ".join(str(i) for i in project_name_list))
    #Contract Name
    contract_name = find(contract_name_dict, no_space_text)
    #if len(contract_name):
    
    print("Contract Name:")
    print(contract_name)
    #Minimum contract amount
    #Maximum contract amount
    #Consortium members
    member_name = find(member_name_dict, no_space_text)
    if len(member_name):
        member_name.replace('、', '\n')
        
    print("Member' Name:")
    print(member_name)
    
if __name__ == '__main__':
    #prase_all('F:\\FDDC2018data\\round1_train_20180518\\重大合同\\pdf\\', 1)
    path = u'F:\\FDDC2018data\\round1_train_20180518\\重大合同\\pdf\\1518.pdf'
    pdf2TxtManager = CPdf2TxtManager()
    text = pdf2TxtManager.changePdfToText(path)
    party_a_dict = load_file("party_a_re")
    party_b_dict = load_file("party_b_re")
    contract_name_dict = load_file("contract_re")
    project_name_dict = load_file("project_re")
    member_name_dict = load_file("members_re")
    cprefix = load_file("correction_prefix")
    csuffix = load_file("correction_suffix")
    cbrk = load_file("correction_brackets")
    re_analyse(text)

证券代码：300137        证券简称：先河环保        公告编号：2012-059 
河北先河环保科技股份有限公司 
关于广东省实施空气质量新标准监测能力建设采购项目采
购中标提示性公告 
本公司及董事会全体成员保证信息披露内容的真实、准确和完
整，没有虚假记载、误导性陈述或重大遗漏。 
2012 年11月30日，广东省政府采购中心(www.gpcgd.com)发布了
《广东省实施空气质量新标准监测能力建设采购项目采购中标公
告》，公告显示我公司为广东省实施空气质量新标准监测能力建设采
购项目采购(采购编号：GPCGD121115HG238F)中标供应商，中标金额：
人民币1888万元。 
 一、中标项目基本情况  
1、采购项目编号：GPCGD121115HG238F 
2、采购项目名称：广东省实施空气质量新标准监测能力建设采
购项目 
3、采购内容：国家环境空气监测网建设 
序号 
项目名称 
1 臭氧分析仪 
2 一氧化碳分析仪 
3 
PM2.5 分析仪（β 射线法） 
4 数据传输与网络化质控平台软硬件 
1 
备注 
数
进口设
量 
备 
13 是 
13 是 
15 是 
12 否 
 
 
 
 
  
  
 
  
8 
5 二氧化硫分析仪 
6 氮氧化物分析仪 
7 
PM10 分析仪（β 射线法） 
质控设备（动态校准、零气发生器、标气、
阀门等） 
9 气象仪（五参数） 
10 城市环境摄影系统 
配套采样系统、机架、稳压电源等辅助设
备 
12 站房改造 
13 新建站房 
    4、采购人名称：广东省环境监测中心  
11 
13 是 
13 是 
13 是 
9 是 
9 是 
11 否 
9 否  
9 否  
1 否  
    5、中标供应商名称：河北先河环保科技股份有限公司 
6、中标金额：￥18880000 
7、中标公示期：自中标公告发布之日起七个工作日 
二、预中标项目对公司的影响  
本次中标金额占公司 2011 年度经审计的合并会计报表营业收
入的12.88%； 若本次中标项目最终签订合同并如期履行，将对公司
业绩产生影响。 
三、风险提示  
截止公告日，项目尚在中标公示期，因此最终中标结果仍存在不
确定性。项目具体内容、中标金额以最终签署的合同为准。待合同正
式签订后公司将履行相应的信息披露程